<a href="https://colab.research.google.com/github/abhinavgairola/NLP_Timeseries/blob/main/Sec_Edgar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **A webscraper file to parse master file of SEC database. The base idea of the scraper is explained in this youtube link <a href="https://www.youtube.com/watch?v=eut8-iOiJ_Q&list=PLcFcktZ0wnNl5X7Qn1JM4jhrIOBsNj1qa&index=2"> Python-Finance-SEC</a>. However, I have given it my own spin and used beautifulsoup and only extracted the masterfile. The scraper will automatically create yearly and quarterly folders and parse the masterfiles as txt. This can be loaded in pandas using standard pandas functions such as read_table. Additionaly it has the ability to extract the CIK number too.**



In [7]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
#CIK={}&Find=Search&owner=exclude&action=getcompany'
def CIK_extracter(TKS,payload_):
  """
  A method to extract CIK numbers for the company tickers.
  @Input:--Ticker name.
  @Input:---And the payload dictionary with the various attributes for example:
  {'CIK': None, 'Find': 'Search','owner':'exclude','action':'getcompany'}
  @Output:-- CIK number.
  """
  URL = 'http://www.sec.gov/cgi-bin/browse-edgar?'
  CIK = re.compile(r'CIK=(\d\d\d\d\d\d\d\d\d\d)')
  CIK_number = {}
  for tickers in TKS:
    payload_['CIK']=tickers
    html_doc=requests.post(URL, params=payload_).text
    print(requests.post(URL, params=payload_).url)
    soup = BeautifulSoup(html_doc, 'html.parser')
    for link in (soup.findAll('a')):
      results= (re.findall(CIK,link.get('href')))
      if results:
        CIK_number[tickers]=(results[0])


  print(CIK_number)
  return CIK_number

###**A Central Index Key or CIK number is a number given to an individual, company, or foreign government by the United States Securities and Exchange Commission. The number is used to identify its filings in several online databases, including EDGAR. The numbers are ten digits in length.**

In [9]:
Tech_Tickers = ['goog', 'aapl','amzn','msft','fb','nvda','yelp']
payload = {'CIK': None, 'Find': 'Search','owner':'exclude','action':'getcompany'}
CIK_ticks_mapping = CIK_extracter(Tech_Tickers,payload)
CIK_df = pd.DataFrame(CIK_ticks_mapping,index=[0])
CIK_df_T = CIK_df.T.reset_index().rename(columns={'index':'Tickers',0:'CIK'})
CIK_df_T



https://www.sec.gov/cgi-bin/browse-edgar?CIK=goog&Find=Search&owner=exclude&action=getcompany
https://www.sec.gov/cgi-bin/browse-edgar?CIK=aapl&Find=Search&owner=exclude&action=getcompany
https://www.sec.gov/cgi-bin/browse-edgar?CIK=amzn&Find=Search&owner=exclude&action=getcompany
https://www.sec.gov/cgi-bin/browse-edgar?CIK=msft&Find=Search&owner=exclude&action=getcompany
https://www.sec.gov/cgi-bin/browse-edgar?CIK=fb&Find=Search&owner=exclude&action=getcompany
https://www.sec.gov/cgi-bin/browse-edgar?CIK=nvda&Find=Search&owner=exclude&action=getcompany
https://www.sec.gov/cgi-bin/browse-edgar?CIK=yelp&Find=Search&owner=exclude&action=getcompany
{'goog': '0001652044', 'aapl': '0000320193', 'amzn': '0001018724', 'msft': '0000789019', 'fb': '0001326801', 'nvda': '0001045810', 'yelp': '0001345016'}


,Tickers,CIK
0,goog,0001652044
1,aapl,0000320193
2,amzn,0001018724
3,msft,0000789019
4,fb,0001326801
5,nvda,0001045810
6,yelp,0001345016


In [10]:
def url_generator(base_url,comp):
  url = base_url

  for r in comp:
    url = '{}/{}'.format(url,r)
  return url
import urllib, os

In [11]:
def data_downloader(years):
  """
  A completely automatic SEC edgars data masterfile parser. This file contains
  a number of information such as the company name, form file and the url to the
  form.
  @Input:--A list of years for which the master file is needed.
  @output:-- Year and quarter wise folders on google colab with the master file.
  """
  if os.path.isdir('/content/drive/MyDrive/Master_files')==True:
    !rm -r /content/drive/MyDrive/Master_files

  base_url = r'https://www.sec.gov/Archives/edgar/daily-index'
  for year in years:
    link___ = '/content/drive/MyDrive/Master_files/'+year
    if os.path.isdir(link___) == False:
      link__ = '/content/drive/MyDrive/Master_files/{}/'.format(year)
      os.makedirs(link__)
    year_url = url_generator(base_url, [year, 'index.json'])
    year_url
    content = requests.get(year_url).json()
    content
    for item in content['directory']['item']:
      print('='*200)
      link____= '/content/drive/MyDrive/Master_files/{}'.format(year)+'/'+item['name']+'/'
      if os.path.isdir(link____) == False:
        temp_if = item['name']
        link_ = '/content/drive/MyDrive/Master_files/'+year+'/'+temp_if+'/'
        os.makedirs(link_)
      print('Extracting the data for {}'.format(item['name']))

      qtr_url = url_generator(base_url,[year,item['name'],'index.json'])
      print('='*200)
      print(item['name']+' url is {}'.format(qtr_url))
      print('='*200)
      content_qtr = requests.get(qtr_url).json()
      print("Content is {}".format(content_qtr))
      counter = 0
      for file in content_qtr['directory']['item']:
        if (file['name'].partition('.')[0]) == 'master':
          file_url = url_generator(base_url, [year,item['name'],file['name']])
          print(file_url)     
          data = requests.get(file_url)
          print("This is data",type(data))
          open('/content/drive/MyDrive/Master_files/'+year+'/'+item['name']+'/'+file['name'].partition('.')[0]+file['name'].partition('.')[2].partition('.')[0]+'.txt','wb').write(data.content)
          counter +=1

In [ ]:
YEARS = [str(num) for num in range(2005,2016)]
print(YEARS)
data_downloader(YEARS)
#YEARS
#file['name'].replace('.idx',"")

['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
Extracting the data for QTR1
QTR1 url is https://www.sec.gov/Archives/edgar/daily-index/2005/QTR1/index.json
Content is {'directory': {'parent-dir': '../', 'item': [{'last-modified': '01/04/2005 07:01:56 PM', 'size': '826 KB', 'name': 'company.20050103.idx', 'href': 'company.20050103.idx', 'type': 'file'}, {'last-modified': '01/05/2005 07:02:07 PM', 'size': '1517 KB', 'name': 'company.20050104.idx', 'href': 'company.20050104.idx', 'type': 'file'}, {'last-modified': '01/06/2005 07:02:03 PM', 'size': '973 KB', 'name': 'company.20050105.idx', 'href': 'company.20050105.idx', 'type': 'file'}, {'last-modified': '01/07/2005 07:01:57 PM', 'size': '476 KB', 'name': 'company.20050106.idx', 'href': 'company.20050106.idx', 'type': 'file'}, {'last-modified': '01/10/2005 07:02:01 PM', 'size': '537 KB', 'name': 'company.20050107.idx', 'href': 'company.20050107.idx', 'type': 'file'}, {'last-modified': '01/11/2005 

In [ ]:
#!rm -r /content/drive/MyDrive/Master_files
file['name'].partition('.')[0]+'.csv'
#df
'/content/drive/MyDrive/Master_files/'+year+'/'+item['name']+'/'+file['name'].partition('.')[0]+file['name'].partition('.')[2].partition('.')[0]+'.csv'